<a href="https://colab.research.google.com/github/fyprjgroup3/FY_project/blob/avik's_work/Labelizer_nb_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checking GPU

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16865781131516869182, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5606414083037656285
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12893431869631054757
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330115994
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4136631872885787359
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

# Getting Dataset & Required Libraries

In [4]:
import pandas as pd
import numpy as np
import re
import json
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import string
import collections
#nltk.download('stopwords')

from google.colab import drive
from google.colab import files
#import io
#uploaded = files.upload()

drive.mount('/content/drive') # mounting drive in each runtime

pd.set_option("display.max_columns",2500)
pd.set_option("display.max_rows",2500)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
#import the data
data=pd.concat([pd.read_csv('/content/drive/My Drive/FY_project/dataset/results-20190513-ofapr19.csv'), 
                pd.read_csv('/content/drive/My Drive/FY_project/dataset/results-20190613-ofmay19.csv'), 
                pd.read_csv('/content/drive/My Drive/FY_project/dataset/results-20200415-ofjun19.csv'),
                pd.read_csv('/content/drive/My Drive/FY_project/dataset/results-apr-questions.csv'),
                pd.read_csv('/content/drive/My Drive/FY_project/dataset/results-jul-questions.csv'),
                pd.read_csv('/content/drive/My Drive/FY_project/dataset/results-bug_20k.csv'),
                pd.read_csv('/content/drive/My Drive/FY_project/dataset/results-enhancement_17k.csv')])
#data=pd.read_csv('E:\yearproject.csv')
data.tail()

,action,url,title,body,labels
16995,"""closed""","""https://github.com/RagtagOpen/freefrom-compen...","""Footer should match prototype""","""Footer should match prototype (see [here](htt...","[{""id"":1446836041,""node_id"":""MDU6TGFiZWwxNDQ2O..."
16996,"""closed""","""https://github.com/frc5587/TitanBot/issues/7""","""\""Getting Started\"" documentation""","""In accordance with #6, all sensitive informat...","[{""id"":1515855500,""node_id"":""MDU6TGFiZWwxNTE1O..."
16997,"""closed""","""https://github.com/Jaspero/jms/issues/108""","""Advanced options for Table""","""- [ ] Search\r\n- [x] Filter\r\n- [x] Hiding ...","[{""id"":1366968101,""node_id"":""MDU6TGFiZWwxMzY2O..."
16998,"""closed""","""https://github.com/WeBankPartners/wecube-plug...","""视图配置-列表选择指标后，改变指标表达式造成删除指标项无法与选择框选项联动""","""**描述您的需求** \r\n如题\r\n\r\n\r\n**预期行为** \r\n上...","[{""id"":1503246165,""node_id"":""MDU6TGFiZWwxNTAzM..."
16999,"""closed""","""https://github.com/ICIJ/datashare/issues/287""","""Create a staging""","""""","[{""id"":361147909,""node_id"":""MDU6TGFiZWwzNjExND..."


In [0]:
from sklearn.utils import shuffle
data=shuffle(data)

In [7]:
print("No. of rows:",data.shape[0])

No. of rows: 74620


# Data Cleaning

In [8]:
print('No. of duplicate rows: ',data.duplicated().sum())

No. of duplicate rows:  657


In [0]:
data.drop_duplicates(inplace=True)

In [10]:
data.shape

(73963, 5)

In [11]:
# select necessary features
data=data[["title","body","labels"]]
data.head()

,title,body,labels
8919,"""Palm oil detection needs to be made global an...","""http://world.openfoodfacts.org/ingredients-th...","[{""id"":523779560,""node_id"":""MDU6TGFiZWw1MjM3Nz..."
1554,"""Simple Celsius to Fahrenheit Model Fails""","""**Describe the bug**\r\nNot sure what I am do...","[{""id"":1174068775,""node_id"":""MDU6TGFiZWwxMTc0M..."
5801,"""Is BACKGRND keyword needed/used in multiexpos...","""The data model schema `multiexposure.schema.y...","[{""id"":390252307,""node_id"":""MDU6TGFiZWwzOTAyNT..."
6044,"""diff: Stats reported by dvc not correct""","""# Description\r\n\r\nWhen tracking a director...","[{""id"":552925963,""node_id"":""MDU6TGFiZWw1NTI5Mj..."
10996,"""Incompatibility between ButterFaces and Boots...","""Twitter user Jonathan LALOU ‏(@John_the_Cowbo...","[{""id"":135207448,""node_id"":""MDU6TGFiZWwxMzUyMD..."


In [12]:
# remove rows with empty labels
def clean(s):
    if s=="[]":
        s=np.nan
    return s
    
data['labels']=data['labels'].apply(lambda x: clean(x))
data.head()

,title,body,labels
8919,"""Palm oil detection needs to be made global an...","""http://world.openfoodfacts.org/ingredients-th...","[{""id"":523779560,""node_id"":""MDU6TGFiZWw1MjM3Nz..."
1554,"""Simple Celsius to Fahrenheit Model Fails""","""**Describe the bug**\r\nNot sure what I am do...","[{""id"":1174068775,""node_id"":""MDU6TGFiZWwxMTc0M..."
5801,"""Is BACKGRND keyword needed/used in multiexpos...","""The data model schema `multiexposure.schema.y...","[{""id"":390252307,""node_id"":""MDU6TGFiZWwzOTAyNT..."
6044,"""diff: Stats reported by dvc not correct""","""# Description\r\n\r\nWhen tracking a director...","[{""id"":552925963,""node_id"":""MDU6TGFiZWw1NTI5Mj..."
10996,"""Incompatibility between ButterFaces and Boots...","""Twitter user Jonathan LALOU ‏(@John_the_Cowbo...","[{""id"":135207448,""node_id"":""MDU6TGFiZWwxMzUyMD..."


In [0]:
data.dropna(inplace=True)

In [14]:
data.head()

,title,body,labels
8919,"""Palm oil detection needs to be made global an...","""http://world.openfoodfacts.org/ingredients-th...","[{""id"":523779560,""node_id"":""MDU6TGFiZWw1MjM3Nz..."
1554,"""Simple Celsius to Fahrenheit Model Fails""","""**Describe the bug**\r\nNot sure what I am do...","[{""id"":1174068775,""node_id"":""MDU6TGFiZWwxMTc0M..."
5801,"""Is BACKGRND keyword needed/used in multiexpos...","""The data model schema `multiexposure.schema.y...","[{""id"":390252307,""node_id"":""MDU6TGFiZWwzOTAyNT..."
6044,"""diff: Stats reported by dvc not correct""","""# Description\r\n\r\nWhen tracking a director...","[{""id"":552925963,""node_id"":""MDU6TGFiZWw1NTI5Mj..."
10996,"""Incompatibility between ButterFaces and Boots...","""Twitter user Jonathan LALOU ‏(@John_the_Cowbo...","[{""id"":135207448,""node_id"":""MDU6TGFiZWwxMzUyMD..."


In [15]:
print("No. of rows:",data.shape[0])

No. of rows: 61957


In [0]:
data.reset_index(inplace=True,drop=True)

In [17]:
# check format of labels
data['labels'][:5]

0    [{"id":523779560,"node_id":"MDU6TGFiZWw1MjM3Nz...
1    [{"id":1174068775,"node_id":"MDU6TGFiZWwxMTc0M...
2    [{"id":390252307,"node_id":"MDU6TGFiZWwzOTAyNT...
3    [{"id":552925963,"node_id":"MDU6TGFiZWw1NTI5Mj...
4    [{"id":135207448,"node_id":"MDU6TGFiZWwxMzUyMD...
Name: labels, dtype: object

In [18]:
data['labels'][3]

'[{"id":552925963,"node_id":"MDU6TGFiZWw1NTI5MjU5NjM=","url":"https://api.github.com/repos/iterative/dvc/labels/bug","name":"bug","color":"f61414","default":true},{"id":1280836913,"node_id":"MDU6TGFiZWwxMjgwODM2OTEz","url":"https://api.github.com/repos/iterative/dvc/labels/p2-medium","name":"p2-medium","color":"cc0000","default":false}]'

In [19]:
#since labels are of string type, convert into list of dictionaries
json.loads(data['labels'][3])

[{'color': 'f61414',
  'default': True,
  'id': 552925963,
  'name': 'bug',
  'node_id': 'MDU6TGFiZWw1NTI5MjU5NjM=',
  'url': 'https://api.github.com/repos/iterative/dvc/labels/bug'},
 {'color': 'cc0000',
  'default': False,
  'id': 1280836913,
  'name': 'p2-medium',
  'node_id': 'MDU6TGFiZWwxMjgwODM2OTEz',
  'url': 'https://api.github.com/repos/iterative/dvc/labels/p2-medium'}]

In [20]:
#extract only name field from labels
[x['name'] for x in json.loads(data['labels'][3])]

['bug', 'p2-medium']

In [21]:
#convert into list of lists
labels=[]
for i in data['labels']:
    labels.append([x['name'] for x in json.loads(i)])
labels[:5]

[['P3', 'blocking mobile apps', 'bug', 'palm oil'],
 ['question'],
 ['datamodels', 'question'],
 ['bug', 'p2-medium'],
 ['bug', 'issue with other JSF fwk', 'verifying issue']]

In [22]:
data['labels']=labels
data.head()

,title,body,labels
0,"""Palm oil detection needs to be made global an...","""http://world.openfoodfacts.org/ingredients-th...","[P3, blocking mobile apps, bug, palm oil]"
1,"""Simple Celsius to Fahrenheit Model Fails""","""**Describe the bug**\r\nNot sure what I am do...",[question]
2,"""Is BACKGRND keyword needed/used in multiexpos...","""The data model schema `multiexposure.schema.y...","[datamodels, question]"
3,"""diff: Stats reported by dvc not correct""","""# Description\r\n\r\nWhen tracking a director...","[bug, p2-medium]"
4,"""Incompatibility between ButterFaces and Boots...","""Twitter user Jonathan LALOU ‏(@John_the_Cowbo...","[bug, issue with other JSF fwk, verifying issue]"


In [0]:
# function for text cleaning 
def clean_text(s):
    s=re.sub('\\\\r',' ',s)  #remove \r
    s=re.sub('[^a-zA-Z]',' ',s) #remove everything but letters
    s=''.join([char for char in s if char not in '!"\',?']) #remove all punctuations
    s=' '.join(s.split()) # remove whitespaces 
    s=s.lower()# convert text to lowercase 
    return s

In [0]:
# clean title and body fields
data['title'] = data['title'].apply(lambda x: clean_text(x))
data['body'] = data['body'].apply(lambda x: clean_text(x))

In [25]:
data.head(10)

,title,body,labels
0,palm oil detection needs to be made global and...,http world openfoodfacts org ingredients that ...,"[P3, blocking mobile apps, bug, palm oil]"
1,simple celsius to fahrenheit model fails,describe the bug nnot sure what i am doing wro...,[question]
2,is backgrnd keyword needed used in multiexposu...,the data model schema multiexposure schema yam...,"[datamodels, question]"
3,diff stats reported by dvc not correct,description n nwhen tracking a directory using...,"[bug, p2-medium]"
4,incompatibility between butterfaces and bootsf...,twitter user jonathan lalou john the cowboy re...,"[bug, issue with other JSF fwk, verifying issue]"
5,germanized klarna,i m using woocommerce germanized pro and tryin...,[has fix]
6,keycloak migrate to new version e g,for security reasons it is always positive to ...,[feature]
7,sim won t open in ie,for https github com phetsims qa issues nthe m...,"[status:ready-for-review, type:bug]"
8,invalid integration doc for symfony,hi guys n nin your doc described integration w...,"[question, wontfix]"
9,nullpointerexception when using otf font type ...,the font naste is a otf font with non cid n nw...,"[bug, help wanted]"


In [0]:
data['merged']=data['title']+" "+data['body']

In [27]:
data[['title','merged']]

,title,merged
0,palm oil detection needs to be made global and...,palm oil detection needs to be made global and...
1,simple celsius to fahrenheit model fails,simple celsius to fahrenheit model fails descr...
2,is backgrnd keyword needed used in multiexposu...,is backgrnd keyword needed used in multiexposu...
3,diff stats reported by dvc not correct,diff stats reported by dvc not correct descrip...
4,incompatibility between butterfaces and bootsf...,incompatibility between butterfaces and bootsf...
...,...,...
61952,this quit working yesterday,this quit working yesterday n n n warning n ig...
61953,d placer l instance mobivlm vers neptune,d placer l instance mobivlm vers neptune actue...
61954,connect to organizations search api,connect to organizations search api is your fe...
61955,email newsletter sign up submit button doesn t...,email newsletter sign up submit button doesn t...


In [0]:
#Labels of different issues are in different forms as they are created manually
#Need to bring all in same shape to get proper count

def clean_label(l):
    s=','.join(l) # convert list to string
    s=re.sub(r'\[[^\]]*\]','',s) # remove everything within []
    s=re.sub('[^a-zA-Z\,]',' ',s) # remove everything but letters and comma
    s=' '.join(s.split()) # remove extra spaces
    s=s.lower() # to lowercase
    lst=s.split(',')
    return [lb.strip() for lb in lst] #strip spaces

In [0]:
data['clean_lbl']=data['labels'].apply(lambda x: clean_label(x))

In [30]:
data.head()

,title,body,labels,merged,clean_lbl
0,palm oil detection needs to be made global and...,http world openfoodfacts org ingredients that ...,"[P3, blocking mobile apps, bug, palm oil]",palm oil detection needs to be made global and...,"[p, blocking mobile apps, bug, palm oil]"
1,simple celsius to fahrenheit model fails,describe the bug nnot sure what i am doing wro...,[question],simple celsius to fahrenheit model fails descr...,[question]
2,is backgrnd keyword needed used in multiexposu...,the data model schema multiexposure schema yam...,"[datamodels, question]",is backgrnd keyword needed used in multiexposu...,"[datamodels, question]"
3,diff stats reported by dvc not correct,description n nwhen tracking a directory using...,"[bug, p2-medium]",diff stats reported by dvc not correct descrip...,"[bug, p medium]"
4,incompatibility between butterfaces and bootsf...,twitter user jonathan lalou john the cowboy re...,"[bug, issue with other JSF fwk, verifying issue]",incompatibility between butterfaces and bootsf...,"[bug, issue with other jsf fwk, verifying issue]"


In [0]:
#get all labels in a single list
l=[]
for i in data['clean_lbl']:
    l.append(i)

In [32]:
l[:5]

[['p', 'blocking mobile apps', 'bug', 'palm oil'],
 ['question'],
 ['datamodels', 'question'],
 ['bug', 'p medium'],
 ['bug', 'issue with other jsf fwk', 'verifying issue']]

In [0]:
#Count frequency of labels
count={}
for y in l:  
    for x in y:
        if x not in count:
            count[x]=1
        else:
            count[x]+=1
d = collections.Counter(count)
freq=[]
for w,c in d.most_common(2502):
    freq.append([w,c])

In [34]:
#top 50 frequent labels
freq[:50]

[['bug', 19221],
 ['enhancement', 15414],
 ['question', 11555],
 ['help wanted', 1812],
 ['severity', 1667],
 ['feature', 1550],
 ['type bug', 1267],
 ['good first issue', 1217],
 ['triaged', 923],
 ['kind bug', 883],
 ['feature request', 747],
 ['wontfix', 732],
 ['stale', 689],
 ['product question', 675],
 ['', 661],
 ['engine gecko', 595],
 ['cxp', 573],
 ['type question', 551],
 ['p', 533],
 ['in progress', 463],
 ['type enhancement', 458],
 ['documentation', 428],
 ['fixed', 331],
 ['assigned to author', 331],
 ['priority high', 330],
 ['duplicate', 326],
 ['type feature', 304],
 ['doc enhancement', 303],
 ['invalid', 291],
 ['pri', 268],
 ['lifecycle rotten', 259],
 ['priority p', 254],
 ['kind feature', 241],
 ['priority medium', 236],
 ['hacktoberfest', 230],
 ['new feature', 223],
 ['frontend', 221],
 ['doc bug', 216],
 ['high priority', 213],
 ['browser firefox', 209],
 ['kind question', 209],
 ['browser firefox mobile', 204],
 ['v', 202],
 ['priority critical', 198],
 ['ui',

In [0]:
#"type bug","kind bug" & "bug fix" are same, need to make them all as "bug"
#Same thing applies for other labels. Using multiple string replaces for the same purpose
lbls=[]
for i in l:
    i=[w.replace('type bug','bug').replace('kind bug','bug').replace('bug fix','bug').replace('bug bug','bug').replace('t bug','bug').replace('c bug','bug')
       .replace('c enhancement','enhancement').replace('doenhancement','enhancement').replace('type enhancement','enhancement')
       .replace('kind enhancement','enhancement').replace('improvement','enhancement').replace('suggestion','enhancement')
       .replace('t be enhancement','enhancement').replace('t fe enhancement','enhancement')
       .replace('feature request','feature').replace('new feature','feature').replace('type feature','feature')
       .replace('kind feature','feature').replace('feature refactoring','feature')
       .replace('priority high','highpriority').replace('priority critical','highpriority').replace('priority p','highpriority')
       .replace('high priority','highpriority').replace('p high','highpriority').replace('priority important','highpriority')
       .replace('priority medium','mediumpriority').replace('medium priority','mediumpriority')
       .replace('priority med','mediumpriority').replace('p medium','mediumpriority')
       .replace('low priority','lowpriority').replace('priority low','lowpriority')
       .replace('type question','question').replace('kind question','question').replace('product question','question')
       .replace('type support','support').replace('help wanted','helpwanted')
       .replace('docs','documentation').replace('documentation needed','documentation').replace('type documentation','documentation')
       .replace('type task','task').replace('front end','frontend')
       .replace('type helpwanted','helpwanted').replace('in review','review').replace('triaged','triage') for w in i]
    lbls.append(i)

In [0]:
count={}
for y in lbls:
    for x in y:
        if x not in count:
            count[x]=1
        else:
            count[x]+=1
d = collections.Counter(count)
freq=[]
for w,c in d.most_common():
    freq.append([w,c])

In [37]:
freq[:50]

[['bug', 21798],
 ['enhancement', 16491],
 ['question', 12990],
 ['feature', 3239],
 ['helpwanted', 1821],
 ['severity', 1667],
 ['highpriority', 1228],
 ['good first issue', 1217],
 ['triage', 995],
 ['wontfix', 732],
 ['stale', 689],
 ['', 661],
 ['engine gecko', 595],
 ['documentation', 591],
 ['cxp', 573],
 ['p', 533],
 ['in progress', 463],
 ['mediumpriority', 452],
 ['fixed', 331],
 ['assigned to author', 331],
 ['duplicate', 326],
 ['lowpriority', 309],
 ['invalid', 291],
 ['frontend', 281],
 ['pri', 268],
 ['lifecycle rotten', 259],
 ['hacktoberfest', 230],
 ['dobug', 216],
 ['browser firefox', 209],
 ['browser firefox mobile', 204],
 ['v', 202],
 ['ui', 198],
 ['priority', 170],
 ['priority normal', 158],
 ['needs triage', 156],
 ['backend', 152],
 ['browser fenix', 140],
 ['task', 138],
 ['greenkeeper', 138],
 ['discussion', 132],
 ['api', 130],
 ['answered', 129],
 ['active directory svc', 123],
 ['done', 116],
 ['core', 114],
 ['needs more info', 114],
 ['review', 111],
 ['

In [0]:
#feature and enhancement are same hence using single word enhancement
lbls1=[]
for i in lbls:
    i=[w.replace('feature','enhancement')
        for w in i]
    lbls1.append(i)

In [0]:
count={}
for y in lbls1:
    for x in y:
        if x not in count:
            count[x]=1
        else:
            count[x]+=1
d = collections.Counter(count)
freq=[]
for w,c in d.most_common():
    freq.append([w,c])

In [40]:
freq[:50]

[['bug', 21798],
 ['enhancement', 19730],
 ['question', 12990],
 ['helpwanted', 1821],
 ['severity', 1667],
 ['highpriority', 1228],
 ['good first issue', 1217],
 ['triage', 995],
 ['wontfix', 732],
 ['stale', 689],
 ['', 661],
 ['engine gecko', 595],
 ['documentation', 591],
 ['cxp', 573],
 ['p', 533],
 ['in progress', 463],
 ['mediumpriority', 452],
 ['fixed', 331],
 ['assigned to author', 331],
 ['duplicate', 326],
 ['lowpriority', 309],
 ['invalid', 291],
 ['frontend', 281],
 ['pri', 268],
 ['lifecycle rotten', 259],
 ['hacktoberfest', 230],
 ['dobug', 216],
 ['browser firefox', 209],
 ['browser firefox mobile', 204],
 ['v', 202],
 ['ui', 198],
 ['priority', 170],
 ['priority normal', 158],
 ['needs triage', 156],
 ['backend', 152],
 ['browser fenix', 140],
 ['task', 138],
 ['greenkeeper', 138],
 ['discussion', 132],
 ['api', 130],
 ['answered', 129],
 ['active directory svc', 123],
 ['done', 116],
 ['core', 114],
 ['needs more info', 114],
 ['review', 111],
 ['support', 110],
 ['d

In [41]:
#add cleaned labels to the dataset
data['lbls_clean']=lbls1
data.head()

,title,body,labels,merged,clean_lbl,lbls_clean
0,palm oil detection needs to be made global and...,http world openfoodfacts org ingredients that ...,"[P3, blocking mobile apps, bug, palm oil]",palm oil detection needs to be made global and...,"[p, blocking mobile apps, bug, palm oil]","[p, blocking mobile apps, bug, palm oil]"
1,simple celsius to fahrenheit model fails,describe the bug nnot sure what i am doing wro...,[question],simple celsius to fahrenheit model fails descr...,[question],[question]
2,is backgrnd keyword needed used in multiexposu...,the data model schema multiexposure schema yam...,"[datamodels, question]",is backgrnd keyword needed used in multiexposu...,"[datamodels, question]","[datamodels, question]"
3,diff stats reported by dvc not correct,description n nwhen tracking a directory using...,"[bug, p2-medium]",diff stats reported by dvc not correct descrip...,"[bug, p medium]","[bug, mediumpriority]"
4,incompatibility between butterfaces and bootsf...,twitter user jonathan lalou john the cowboy re...,"[bug, issue with other JSF fwk, verifying issue]",incompatibility between butterfaces and bootsf...,"[bug, issue with other jsf fwk, verifying issue]","[bug, issue with other jsf fwk, verifying issue]"


In [0]:
#Not all labels can be taken for model building hence selecting
#Important and some common labels of almost every repository
selected_labels=['bug','enhancement','question']
#,'helpwanted','documentation','task','invalid','support','review','frontend'

#Not all labels can be taken for model building
#Selecting top 50 frequent labels
#selected_labels=[]
#for i in freq[:50]:
#    selected_labels.append(i[0])

In [0]:
#selected_labels[:5]

In [44]:
dset=data[["merged","lbls_clean"]]
dset.head()

,merged,lbls_clean
0,palm oil detection needs to be made global and...,"[p, blocking mobile apps, bug, palm oil]"
1,simple celsius to fahrenheit model fails descr...,[question]
2,is backgrnd keyword needed used in multiexposu...,"[datamodels, question]"
3,diff stats reported by dvc not correct descrip...,"[bug, mediumpriority]"
4,incompatibility between butterfaces and bootsf...,"[bug, issue with other jsf fwk, verifying issue]"


In [45]:
#Keep only selected labels and remove others
#each row is intersected with the selected_labels lists
def intersection(lst1, lst2): 
    temp = set(lst2) 
    lst3 = [value for value in lst1 if value in temp] 
    return lst3

dset['lbls_clean']=[intersection(selected_labels,i) for i in dset['lbls_clean']]
dset.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,merged,lbls_clean
0,palm oil detection needs to be made global and...,[bug]
1,simple celsius to fahrenheit model fails descr...,[question]
2,is backgrnd keyword needed used in multiexposu...,[question]
3,diff stats reported by dvc not correct descrip...,[bug]
4,incompatibility between butterfaces and bootsf...,[bug]
5,germanized klarna i m using woocommerce german...,[]
6,keycloak migrate to new version e g for securi...,[enhancement]
7,sim won t open in ie for https github com phet...,[bug]
8,invalid integration doc for symfony hi guys n ...,[question]
9,nullpointerexception when using otf font type ...,[bug]


In [0]:
#Selected labels resulted in some empty labels as many repositories do not have them
def clean(s):
    if len(s)==0:
        s=np.nan
    return s

ldata=dset.copy(deep=True)   
ldata['lbls_clean']=ldata['lbls_clean'].apply(lambda x: clean(x))
ldata.dropna(inplace=True) #Removing empty-label rows

In [47]:
ldata.shape

(52363, 2)

In [0]:
#rows having merged body and title with empty values are removed
#ldata=ldata[ldata['body']!=""]
#ldata=ldata[ldata['title']!=""]
ldata=ldata[ldata['merged']!=""]

In [49]:
#finally dataset is reduced to 52,363 rows after cleaning and pre-processing
ldata.shape

(52363, 2)

In [0]:
ldata.reset_index(inplace=True,drop=True)

In [51]:
ldata.head()

,merged,lbls_clean
0,palm oil detection needs to be made global and...,[bug]
1,simple celsius to fahrenheit model fails descr...,[question]
2,is backgrnd keyword needed used in multiexposu...,[question]
3,diff stats reported by dvc not correct descrip...,[bug]
4,incompatibility between butterfaces and bootsf...,[bug]


In [0]:
#one_hot_encode the labels
def one_hot_encode(data,label):
  lb=[]
  for row in data:
    if label in row:
      lb.append(1)
    else:
      lb.append(0)
  return lb

In [0]:
ldata['bug']=one_hot_encode(ldata['lbls_clean'], 'bug')

In [0]:
ldata['enhancement']=one_hot_encode(ldata['lbls_clean'], 'enhancement')

In [0]:
ldata['question']=one_hot_encode(ldata['lbls_clean'], 'question')

In [0]:
ldata['id']=ldata.index

In [0]:
ldata=ldata[["id","merged", "bug", "enhancement", "question"]]

In [58]:
ldata

,id,merged,bug,enhancement,question
0,0,palm oil detection needs to be made global and...,1,0,0
1,1,simple celsius to fahrenheit model fails descr...,0,0,1
2,2,is backgrnd keyword needed used in multiexposu...,0,0,1
3,3,diff stats reported by dvc not correct descrip...,1,0,0
4,4,incompatibility between butterfaces and bootsf...,1,0,0
...,...,...,...,...,...
52358,52358,this quit working yesterday n n n warning n ig...,0,0,1
52359,52359,d placer l instance mobivlm vers neptune actue...,0,1,0
52360,52360,connect to organizations search api is your fe...,0,1,0
52361,52361,email newsletter sign up submit button doesn t...,1,0,0


# Preparing for BERT

In [0]:
import tensorflow_hub as hub

In [60]:
##install bert if not already done
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 1.9MB/s 


In [61]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [0]:
##use downloaded model, change path accordingly
BERT_VOCAB= '/content/drive/My Drive/FY_project/BERT_models/uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = '/content/drive/My Drive/FY_project/BERT_models/uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = '/content/drive/My Drive/FY_project/BERT_models/uncased_L-12_H-768_A-12/bert_config.json'

In [63]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [64]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [0]:
x_train=ldata[:41890]
x_val=ldata[41890:]

In [66]:
x_train

,id,merged,bug,enhancement,question
0,0,palm oil detection needs to be made global and...,1,0,0
1,1,simple celsius to fahrenheit model fails descr...,0,0,1
2,2,is backgrnd keyword needed used in multiexposu...,0,0,1
3,3,diff stats reported by dvc not correct descrip...,1,0,0
4,4,incompatibility between butterfaces and bootsf...,1,0,0
...,...,...,...,...,...
41885,41885,sorting by size doesn t work for large files e...,1,0,0
41886,41886,as a user i can follow other users so that i c...,0,1,0
41887,41887,crash on adding bookmark nthread multivnc rece...,1,0,0
41888,41888,jvm java java jvm java,0,0,1


In [0]:
ID = 'id'
DATA_COLUMN = 'merged'
LABEL_COLUMNS = ['bug','enhancement','question']

# Model Architecture

In [0]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [0]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values):
        guid = row[0]
        text_a = row[1]
        if labels_available:
            labels = row[2:]
        else:
            labels = [0,0,0]
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [0]:
train_examples = create_examples(x_train)

In [0]:
def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [0]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """
    
    
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([3], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Training

In [0]:
import os
train_file = os.path.join('/content/drive/My Drive/FY_project/working', "train.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

In [77]:
%%time
file_based_convert_examples_to_features(
            train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 41890
INFO:tensorflow:  Batch size = 32
INFO:tensorflow:  Num steps = 1309
CPU times: user 1min 44s, sys: 199 ms, total: 1min 44s
Wall time: 1min 46s


In [0]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

In [0]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [0]:
OUTPUT_DIR = "/content/drive/My Drive/FY_project/working/output"
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [81]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= len(LABEL_COLUMNS),
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/FY_project/working/output', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f50ab258860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [82]:
%%time
print(f'Beginning Training!')
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved.
CPU times: user 5.63 ms, sys: 8 ms, total: 13.6 ms
Wall time: 2.62 s


In [0]:
#save model
#from sklearn.externals import joblib
#joblib.dump(estimator, 'bert_estimator.pkl') 

# Validation

In [0]:
eval_file = os.path.join('/content/drive/My Drive/FY_project/working', "eval.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(x_val)
file_based_convert_examples_to_features(
    eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

In [85]:
# This tells the estimator to run through the entire set.
%%time
eval_steps = None

eval_drop_remainder = False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.



Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:num_labels:3;logits:Tensor("loss/BiasAdd:0", shape=(?, 3), dtype=float32);labels:Tensor("loss/Cast:0",

In [86]:
output_eval_file = os.path.join("/content/drive/My Drive/FY_project/working", "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:***** Eval results *****
INFO:tensorflow:  0 = 0.93622464
INFO:tensorflow:  1 = 0.934296
INFO:tensorflow:  2 = 0.90410745
INFO:tensorflow:  eval_loss = 0.31589803
INFO:tensorflow:  global_step = 1309
INFO:tensorflow:  loss = 0.31570876


# Testing

In [87]:
x_test=x_val.drop(['bug','enhancement','question'], axis=1)
x_test

,id,merged
41890,41890,is there opportunity to compress data before s...
41891,41891,add all filter in date picker provide a genera...
41892,41892,create results page create the index html that...
41893,41893,illegalargumentexception when generating the w...
41894,41894,warn if node api is too high newer nodes are g...
...,...,...
52358,52358,this quit working yesterday n n n warning n ig...
52359,52359,d placer l instance mobivlm vers neptune actue...
52360,52360,connect to organizations search api is your fe...
52361,52361,email newsletter sign up submit button doesn t...


In [0]:
#x_test = test[:10000] #testing a small sample
x_test = x_test.reset_index(drop=True)
test_file = os.path.join('/content/drive/My Drive/FY_project/working', "test.tf_record")
#filename = Path(train_file)
if not os.path.exists(test_file):
    open(test_file, 'w').close()

test_examples = create_examples(x_test, False)

In [89]:
%%time
file_based_convert_examples_to_features(
    test_examples, MAX_SEQ_LENGTH, tokenizer, test_file)

CPU times: user 26.5 s, sys: 31.9 ms, total: 26.5 s
Wall time: 26.7 s


def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [0]:
predict_input_fn = file_based_input_fn_builder(
    input_file=test_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [91]:
%%time
print('Begin predictions!')
predictions = estimator.predict(predict_input_fn)

Begin predictions!
CPU times: user 957 µs, sys: 0 ns, total: 957 µs
Wall time: 1.92 ms


In [0]:
def create_output(predictions):
    probabilities = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction["probabilities"]
        probabilities.append(preds)
    dff = pd.DataFrame(probabilities)
    dff.columns = LABEL_COLUMNS
    
    return dff

In [93]:
output_df = create_output(predictions)
merged_df =  pd.concat([x_test, output_df], axis=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:num_labels:3;logits:Tensor("loss/BiasAdd:0", shape=(?, 3), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
mode: infer probabilities: Tensor("loss/Sigmoid:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/FY_project/working/output/model.ckpt-1309
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [94]:
merged_df

,id,merged,bug,enhancement,question
0,41890,is there opportunity to compress data before s...,0.018923,0.175471,0.880884
1,41891,add all filter in date picker provide a genera...,0.065821,0.881179,0.066191
2,41892,create results page create the index html that...,0.022972,0.959067,0.038789
3,41893,illegalargumentexception when generating the w...,0.735928,0.038010,0.306455
4,41894,warn if node api is too high newer nodes are g...,0.122918,0.881726,0.034691
...,...,...,...,...,...
10468,52358,this quit working yesterday n n n warning n ig...,0.101201,0.046345,0.889630
10469,52359,d placer l instance mobivlm vers neptune actue...,0.326333,0.486993,0.255535
10470,52360,connect to organizations search api is your fe...,0.014757,0.960903,0.060229
10471,52361,email newsletter sign up submit button doesn t...,0.854788,0.146949,0.054254


In [96]:
x_val

,id,merged,bug,enhancement,question
41890,41890,is there opportunity to compress data before s...,0,1,0
41891,41891,add all filter in date picker provide a genera...,0,1,0
41892,41892,create results page create the index html that...,0,1,0
41893,41893,illegalargumentexception when generating the w...,1,0,0
41894,41894,warn if node api is too high newer nodes are g...,0,1,0
...,...,...,...,...,...
52358,52358,this quit working yesterday n n n warning n ig...,0,0,1
52359,52359,d placer l instance mobivlm vers neptune actue...,0,1,0
52360,52360,connect to organizations search api is your fe...,0,1,0
52361,52361,email newsletter sign up submit button doesn t...,1,0,0
